In [2]:
import selenium
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import time
import json
from tqdm import tqdm

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains


In [26]:
# 오늘의 집 셀레늄
options = webdriver.ChromeOptions()
options.add_argument('headless')

# 검색 창 페이지
search_url = 'https://store.ohou.se/ranks?type=best&category_id=10000000'
driver = webdriver.Chrome(executable_path = "C:/Users/se686/Desktop/chromedriver_win32/chromedriver.exe")
driver.implicitly_wait(30) 
driver.get(search_url)

# 스크롤 내리기
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # 1초 대기
    time.sleep(1)
    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height



C:\Users\se686\AppData\Local\Temp\ipykernel_11820\1459616300.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "C:/Users/se686/Desktop/chromedriver_win32/chromedriver.exe")


In [25]:
# 링크 추출
product_links = []
product_elements = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[5]/div/div[1]/article/a')
print(product_elements)

product_element = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[5]/div/div/article/a')
print(product_element)

<selenium.webdriver.remote.webelement.WebElement (session="96517ed82696ef4f7acd9903c8095a44", element="e974ff62-b2d5-41ae-9b85-2f4db4b650e4")>
<selenium.webdriver.remote.webelement.WebElement (session="96517ed82696ef4f7acd9903c8095a44", element="e974ff62-b2d5-41ae-9b85-2f4db4b650e4")>


In [10]:
# 다이닝코드 셀레늄
options = webdriver.ChromeOptions()
options.add_argument('headless')

# 검색 창 페이지
search_url = 'https://www.diningcode.com/search.dc?'
driver = webdriver.Chrome(executable_path = "C:/Users/se686/Desktop/chromedriver_win32/chromedriver.exe")
driver.implicitly_wait(10) 
driver.get(search_url)

# '**역 카페' 검색 + Enter
search_element = driver.find_element(By.XPATH, '//*[@id="root"]/div/header/div/div/div[2]/div/input')
search_element.send_keys('성수역 카페' + Keys.ENTER)

# 검색 결과 더보기 클릭!
more_btn_element = driver.find_element(By.XPATH, '//*[@id="root"]/div/main/div[2]/div[3]/div/button')
more_btn_element.click()

# 예외 처리로 카페 리스트 늘리기
try:
    for i in range(10):
        more_element = driver.find_element(By.CLASS_NAME, 'SearchMore.upper')
        more_element.click()
except:
    print('수집 완료')
    pass

# 1 to 100 카페 링크 가져오기
# get_attribute -> html 무언가 얻어오기
cafe_urls = []
soup_source = driver.find_elements(By.XPATH,'/html/body/div/div/div[1]/div[3]/ol/li/div[1]')
for source in soup_source:
    cafe_urls.append('https://www.diningcode.com/profile.php?rid=' + source.get_attribute('id')[5:])

print(cafe_urls, len(cafe_urls))

C:\Users\se686\AppData\Local\Temp\ipykernel_11820\3236284728.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "C:/Users/se686/Desktop/chromedriver_win32/chromedriver.exe")


수집 완료
['https://www.diningcode.com/profile.php?rid=gwMOA9twBJDG', 'https://www.diningcode.com/profile.php?rid=8UZqn4dn5naj', 'https://www.diningcode.com/profile.php?rid=BkUWWoRBmMxW', 'https://www.diningcode.com/profile.php?rid=kvhFvQegWELq', 'https://www.diningcode.com/profile.php?rid=TpMlBoL70Tge', 'https://www.diningcode.com/profile.php?rid=pSodx8SMq4wP', 'https://www.diningcode.com/profile.php?rid=JI0QtfXarLDr', 'https://www.diningcode.com/profile.php?rid=816eBLdd7tDQ', 'https://www.diningcode.com/profile.php?rid=w3vBm0us9VDH', 'https://www.diningcode.com/profile.php?rid=205chuT865Ip', 'https://www.diningcode.com/profile.php?rid=c9A6GfSJXHgn', 'https://www.diningcode.com/profile.php?rid=wM5hbKBtvtCW', 'https://www.diningcode.com/profile.php?rid=juk2SDq6Gu5c', 'https://www.diningcode.com/profile.php?rid=HLnHOjp0kjSr', 'https://www.diningcode.com/profile.php?rid=C4bft2Lj3Gpv', 'https://www.diningcode.com/profile.php?rid=ca0dNWL6brSv', 'https://www.diningcode.com/profile.php?rid=5yJwh

In [15]:
# 수집한 카페 링크로 반복
review = []
score = []
options = webdriver.ChromeOptions()
options.add_argument('headless')

for cafe_url in tqdm(cafe_urls):
    cafe_driver = webdriver.Chrome(executable_path = "C:/Users/se686/Desktop/chromedriver_win32/chromedriver.exe")
    cafe_driver.get(cafe_url)
    time.sleep(1)
    # 리뷰 더보기 클릭!
    try:
        while True:
            review_btn_element = cafe_driver.find_element(By.ID, 'div_more_review')
            review_btn_element.click()
    except:
        pass
    # 카페별 리뷰 크롤링
    reviews = cafe_driver.find_elements(By.CLASS_NAME,'review_contents.btxt')
    for re in reviews:
        review.append(re.text.replace('\n',' '))
    # 평점 불러오기 - BeautifulSoup 이용..?
    soup = BeautifulSoup(cafe_driver.page_source, 'html.parser')
    for star in soup.select('i.star i'):
        score.append(int(star['style'].split(':')[1].split('%')[0]) // 20)

df_reviews = pd.DataFrame(zip(review, score), columns = ['Review', 'Score'])
df_reviews.to_csv('C:/Users/se686/Desktop/2nd_project_reviews/blue_bottle.csv', encoding='utf-8-sig')

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]C:\Users\se686\AppData\Local\Temp\ipykernel_5720\2221704787.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  cafe_driver = webdriver.Chrome(executable_path = "C:/Users/se686/Desktop/chromedriver_win32/chromedriver.exe")
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.71s/it]
